# exe 3.1 p.91
Your task is to correctly assign the weights from an instance of SelfAttention_v2 to
an instance of SelfAttention_v1. To do this, you need to understand the
relationship between the weights in both versions. (Hint: nn.Linear stores the
weight matrix in a transposed form.) After the assignment, you should observe that
both instances produce the same outputs.

In [2]:
import torch.nn as nn
import torch

In [5]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [86]:
torch.manual_seed(1234)

In [87]:
"""
inputs: A(6x3)
weight matrices: WQ(3,2), WK(3,2), WV(3,2) # random init, 3 input embedding size, 2 chosen emdedding dimensions
K,V: K(6,2) = A * WK, V = A * WV
QY[i]: QY[i](1,2) = A[i](1,3) * Q(3,2) # QY is query for element i, i.e attention given to token i by all other tokens
attention scores: S(1,6) = QY[i](1,2) * K.T(2,6)
embedding dim: ED = K.shape[1] # 
attention weights [i]: AW(1,6) = softmax(S/sqrt(ED), dim=-1). aka. softmax on rows. we divide by square root of ED (why ??)
context vector [i]: AW(1,6) * V(6,2) 
"""

'\ninputs: A(6x3)\nweight matrices: WQ(3,2), WK(3,2), WV(3,2) # random init, 3 input embedding size, 2 chosen emdedding dimensions\nK,V: K(6,2) = A * WK, V = A * WV\nQY[i]: QY[i](1,2) = A[i](1,3) * Q(3,2) # QY is query for element i, i.e attention given to token i by all other tokens\nattention scores: S(1,6) = QY[i](1,2) * K.T(2,6)\nembedding dim: ED = K.shape[1] # \nattention weights [i]: AW(1,6) = softmax(S/sqrt(ED), dim=-1). aka. softmax on rows. we divide by square root of ED (why ??)\ncontext vector [i]: AW(1,6) * V(6,2) \n'

In [88]:
class SelfAttentioV1(nn.Module):
    def __init__(self, d_in, d_out) -> None:
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_keys = nn.Parameter(torch.rand(d_in, d_out))
        self.W_values = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x: torch.Tensor):
        queries = x @ self.W_query
        keys = x @ self.W_keys
        values = x @ self.W_values

        attn_score = queries @ keys.T
        attn_weights = torch.softmax(attn_score / keys.shape[-1]**0.5, dim=-1)
        contextvec = attn_weights @ values
        return contextvec

In [89]:
class SelfAttentioV2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False) -> None:
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, qkv_bias)
        self.W_keys = nn.Linear(d_in, d_out, qkv_bias)
        self.W_values = nn.Linear(d_in, d_out, qkv_bias)

    def forward(self, x: torch.Tensor):
        queries = self.W_query(x)
        keys = self.W_keys(x)
        values = self.W_values(x)

        attn_score = queries @ keys.T
        attn_weights = torch.softmax(attn_score / keys.shape[-1]**0.5, dim=-1)
        contextvec = attn_weights @ values
        return contextvec

In [90]:
d_in = inputs.shape[1]
d_out = 2
sav1 = SelfAttentioV1(d_in=d_in, d_out=d_out)
sav2 = SelfAttentioV2(d_in=d_in, d_out=d_out)

In [91]:
sav1.forward(inputs)

tensor([[0.6641, 1.1324],
        [0.6694, 1.1453],
        [0.6691, 1.1446],
        [0.6592, 1.1247],
        [0.6570, 1.1190],
        [0.6632, 1.1333]], grad_fn=<MmBackward0>)

In [92]:
sav2(inputs) # different output to sav1 because different initial weights

tensor([[0.1478, 0.0307],
        [0.1502, 0.0307],
        [0.1502, 0.0308],
        [0.1492, 0.0305],
        [0.1498, 0.0320],
        [0.1491, 0.0299]], grad_fn=<MmBackward0>)

In [93]:
sav2.W_keys.weight

Parameter containing:
tensor([[ 0.5387,  0.2771, -0.0558],
        [-0.0280,  0.3282, -0.4013]], requires_grad=True)

In [94]:
with torch.no_grad():
    sav2.W_query.weight.copy_(sav1.W_query.T) # transposing because linear inverts d_in, d_out
    sav2.W_keys.weight.copy_(sav1.W_keys.T)
    sav2.W_values.weight.copy_(sav1.W_values.T)

In [106]:
sav1.forward(inputs).sum()

tensor(10.7813, grad_fn=<SumBackward0>)

In [107]:
sav2.forward(inputs).sum()

tensor(10.7813, grad_fn=<SumBackward0>)

In [108]:
sav2.forward(inputs).sum().eq(sav1.forward(inputs).sum())

tensor(True)

# exe 3.2 p.105
Change the input arguments for the MultiHeadAttentionWrapper(...,
num_heads=2) call such that the output context vectors are 2-dimensional instead of
4-dimensional while keeping the setting num_heads=2. Hint: You don't have to modify
the class implementation; you just have to change one of the other input arguments

In [109]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        # For inputs where `num_tokens` exceeds `context_length`, this will result in errors
        # in the mask creation further below.
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forward method. 
        #colorize("current x", x)
        #colorize("current keys", self.W_key)
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose. (because batch, we need to pick the correct dimensions to transpose)
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New
        context_vec = attn_weights @ values
        return context_vec


In [110]:
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


In [116]:
m_inputs = torch.stack((inputs, inputs))
m_inputs.shape

torch.Size([2, 6, 3])

In [117]:
context_length = m_inputs.shape[1]
mhw = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2)

In [119]:
mhw.forward(m_inputs) # will return context vector of 4 dims.

tensor([[[ 0.5038,  0.5031, -0.0953, -0.1885],
         [ 0.6263,  0.5726, -0.1220, -0.2307],
         [ 0.6650,  0.5949, -0.1255, -0.2435],
         [ 0.5994,  0.5285, -0.1250, -0.2207],
         [ 0.5741,  0.5122, -0.0639, -0.2031],
         [ 0.5554,  0.4860, -0.0948, -0.1998]],

        [[ 0.5038,  0.5031, -0.0953, -0.1885],
         [ 0.6263,  0.5726, -0.1220, -0.2307],
         [ 0.6650,  0.5949, -0.1255, -0.2435],
         [ 0.5994,  0.5285, -0.1250, -0.2207],
         [ 0.5741,  0.5122, -0.0639, -0.2031],
         [ 0.5554,  0.4860, -0.0948, -0.1998]]], grad_fn=<CatBackward0>)

In [120]:
# answering question, output context vector Z will be 2 dims.
# changes embeding params d_out
context_length = m_inputs.shape[1]
mhw = MultiHeadAttentionWrapper(d_in=d_in,
                                d_out=1,
                                context_length=context_length,
                                dropout=0.0,
                                num_heads=2)

In [121]:
mhw.forward(m_inputs)

tensor([[[-0.1148, -0.0951],
         [-0.2139, -0.2602],
         [-0.2433, -0.3208],
         [-0.2428, -0.2998],
         [-0.1736, -0.3359],
         [-0.2111, -0.3115]],

        [[-0.1148, -0.0951],
         [-0.2139, -0.2602],
         [-0.2433, -0.3208],
         [-0.2428, -0.2998],
         [-0.1736, -0.3359],
         [-0.2111, -0.3115]]], grad_fn=<CatBackward0>)

# exe 3.3 p.112
Using the MultiHeadAttention class, initialize a multi-head attention module that
has the same number of attention heads as the smallest GPT-2 model (12 attention
heads). Also ensure that you use the respective input and output embedding sizes
similar to GPT-2 (768 dimensions). Note that the smallest GPT-2 model supports a
context length of 1024 tokens

In [122]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`, 
        # this will result in errors in the mask creation further below. 
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forwar

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        # calling contigous becuase of transpose operation before
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec


In [123]:
d_in = 768
d_out = 768
num_heads = 12
mha = MultiHeadAttention(d_in=d_in,
                         d_out=d_out,
                         context_length=d_in,
                         dropout=0.0,
                         num_heads=num_heads)

In [127]:
gpt2_input = torch.randn((2,1,768))
gpt2_input

tensor([[[ 0.5950,  0.0020,  0.6558,  ..., -1.8559,  0.4435, -0.4682]],

        [[ 1.2463, -1.6313,  1.6045,  ...,  1.2252,  0.8412,  1.4212]]])

In [129]:
step = mha.forward(gpt2_input)
step

tensor([[[ 0.2079, -0.0523,  0.2395,  ...,  0.3321, -0.0538, -0.3146]],

        [[-0.0594, -0.0500, -0.0887,  ...,  0.1867,  0.2560, -0.1950]]],
       grad_fn=<ViewBackward0>)

In [130]:
step.shape

torch.Size([2, 1, 768])